### Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai import *
from fastai.tabular import * 
from fastai.text import *
from exp.eventlog import *

### Loading Data

In [3]:
path = untar_data(URLs.BPIC_2012)
log = import_xes(path,extensions=False,classifiers=False,schema=False,log_attributes=False)

failed to parse date: 1970-01-01T00:00:00.000+01:00
failed to parse date: 1970-01-01T00:00:00.000+01:00
failed to parse date: 2012-04-23T00:00:00.000+02:00
failed to parse date: 2011-10-01T00:38:44.546+02:00
failed to parse date: 2012-03-14T16:04:54.681+01:00


In [4]:
df = log.events
df.head()

,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00
1,173688,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00
2,173688,2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00
3,173688,3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00
4,173688,4,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00


In [5]:
pd.get_dummies(df['concept:name'])

,A_ACCEPTED,A_ACTIVATED,A_APPROVED,A_CANCELLED,A_DECLINED,A_FINALIZED,A_PARTLYSUBMITTED,A_PREACCEPTED,A_REGISTERED,A_SUBMITTED,...,O_SELECTED,O_SENT,O_SENT_BACK,W_Afhandelen leads,W_Beoordelen fraude,W_Completeren aanvraag,W_Nabellen incomplete dossiers,W_Nabellen offertes,W_Valideren aanvraag,W_Wijzigen contractgegevens
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [6]:
hi = df.groupby("trace_id")["concept:name"].apply(lambda x: " ".join(x))
hi[0]

'A_SUBMITTED A_PARTLYSUBMITTED A_PREACCEPTED W_Completeren aanvraag W_Completeren aanvraag A_ACCEPTED O_SELECTED A_FINALIZED O_CREATED O_SENT W_Nabellen offertes W_Completeren aanvraag W_Nabellen offertes W_Nabellen offertes W_Nabellen offertes W_Nabellen offertes W_Nabellen offertes O_SENT_BACK W_Valideren aanvraag W_Nabellen offertes W_Valideren aanvraag A_REGISTERED A_APPROVED O_ACCEPTED A_ACTIVATED W_Valideren aanvraag'

In [7]:
values = df['concept:name'].unique()
display(values, len(values))

array(['A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED', 'W_Completeren aanvraag', 'A_ACCEPTED', 'O_SELECTED',
       'A_FINALIZED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes', 'O_SENT_BACK', 'W_Valideren aanvraag',
       'A_REGISTERED', 'A_APPROVED', 'O_ACCEPTED', 'A_ACTIVATED', 'O_CANCELLED', 'W_Wijzigen contractgegevens',
       'A_DECLINED', 'A_CANCELLED', 'W_Afhandelen leads', 'O_DECLINED', 'W_Nabellen incomplete dossiers',
       'W_Beoordelen fraude'], dtype=object)

24

### My Preprocessing

### Preprocessing FastAi

In [ ]:
def encode_data(events, trace_attributes, cat_columns, date_columns, num_columns, vocab=None):
    if vocab is None:
        vocab = {}
        
    events = events.copy()
    trace_attributes = trace_attributes.copy()
    
    for d in date_columns:
        if d in list(events): 
            cat, con = add_datepart(events,d,utc=True)
            for i in range(len(cat)):
                if cat[i] not in cat_columns:
                    cat_columns.append(cat[i])
            if con not in num_columns:
                num_columns.append(con)
                
        if d in list(trace_attributes): 
            cat, con = add_datepart(trace_attributes,d,utc=True)
            for i in range(len(cat)):
                if cat[i] not in cat_columns:
                    cat_columns.append(cat[i])
            if con not in num_columns:
                num_columns.append(con)            
        
    for c in cat_columns: 
        if c in vocab.keys():
            proc = vocab[c]
            
            if c in list(events):
                events[c] = proc(events[c])
            if c in list(trace_attributes):
                trace_attributes[c] = proc(trace_attributes[c])
            
        else:
            if c in list(events): 
                proc = CategoryProcessor(default_spec_tok)
                events[c] = proc(events[c])
            if c in list(trace_attributes): 
                proc = CategoryProcessor()
                trace_attributes[c] = proc(trace_attributes[c])

            vocab[c] = proc   
        
    for n in num_columns:
        if n in vocab:
            mean, std = vocab[n]
            
            if n in list(events):
                events[n] = events[n].astype(float)
                events[n] = (events[n]-mean) / (1e-7 + std)
            if n in list(trace_attributes):
                trace_attributes[n] = trace_attributes[n].astype(float)
                trace_attributes[n] = (trace_attributes[n]-mean) / (1e-7 + std)
                
        else:
            if n in list(events): 
                events[n] = events[n].astype(float)
                mean, std = events[n].mean(), events[n].std()
                events[n] = (events[n]-mean) / (1e-7 + std)
            if n in list(trace_attributes): 
                trace_attributes[n] = trace_attributes[n].astype(float)
                mean, std = trace_attributes[n].mean(), trace_attributes[n].std()
                trace_attributes[n] = (trace_attributes[n]-mean) / (1e-7 + std)

            vocab[n] = mean, std
        
    return events, trace_attributes, cat_columns, num_columns, vocab

In [8]:
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()
default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

In [9]:
from collections import OrderedDict

def uniqueify(x, sort=False):
    res = list(OrderedDict.fromkeys(x).keys())
    if sort: res.sort()
    return res

class Processor():
    def process(self, items): return items

class CategoryProcessor(Processor):
    def __init__(self,default_token=None): 
        self.vocab=None
        self.default_token=default_token

    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            self.vocab = uniqueify(items)
            if self.default_token is not None:
                for o in reversed(self.default_token):
                    if o in self.vocab: self.vocab.remove(o)
                    self.vocab.insert(0, o)
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return self.otoi[item]

    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return self.vocab[idx]

In [10]:
user_numericalize=CategoryProcessor(default_spec_tok)
user_numericalize(df['org:resource'])

activity_numericalize=CategoryProcessor(default_spec_tok)
activity_numericalize(df['concept:name']);

In [11]:
%%time

def create_traces(df):
    ll=[]
    trace_ids=[]
    
    for n, g in df.groupby('trace_id'):
        ea = (g['concept:name'])
        eu = (g['org:resource'])
        ll.append([ea, eu])
        trace_ids.append(n)

    df2 = pd.DataFrame(ll, columns=["Activities",'Users'])
    df2.index = trace_ids
    return df2

data = create_traces(df)

CPU times: user 2.71 s, sys: 230 µs, total: 2.71 s
Wall time: 2.71 s


In [12]:
activity_numericalize.deprocess(activity_numericalize(data['Activities'].iloc[0]))

['A_SUBMITTED',
 'A_PARTLYSUBMITTED',
 'A_PREACCEPTED',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'A_ACCEPTED',
 'O_SELECTED',
 'A_FINALIZED',
 'O_CREATED',
 'O_SENT',
 'W_Nabellen offertes',
 'W_Completeren aanvraag',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'O_SENT_BACK',
 'W_Valideren aanvraag',
 'W_Nabellen offertes',
 'W_Valideren aanvraag',
 'A_REGISTERED',
 'A_APPROVED',
 'O_ACCEPTED',
 'A_ACTIVATED',
 'W_Valideren aanvraag']

In [13]:
data

,Activities,Users
173688,0 A_SUBMITTED 1 A_PART...,0 112 1 112 2 112 3 11...
173691,26 A_SUBMITTED 27 A_PART...,26 112 27 112 28 112 29 11...
173694,65 A_SUBMITTED 66 ...,65 112 66 112 67 112 68 ...
173697,124 A_SUBMITTED 125 A_PARTLYSUBMIT...,124 112 125 112 126 112 Name: org:res...
173700,127 A_SUBMITTED 128 A_PARTLYSUBMIT...,127 112 128 112 129 112 Name: org:res...
173703,130 A_SUBMITTED 131 A_PA...,130 112 131 112 132 112 133 ...
173706,139 A_SUBMITTED 140 A_PA...,139 112 140 112 141 112 142 ...
173709,153 A_SUBMITTED 154 A_PA...,153 112 154 112 155 112 156 ...
173712,165 A_SUBMITTED 166 A_PA...,165 112 166 112 167 112 168 ...
173715,179 A_SUBMITTED 180 A_PA...,179 112 180 112 181 112 182 ...


In [14]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [15]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs * n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        
        # The features
        x = arr[:, n:n+n_steps]
        
        # The targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [16]:
def random_split(np_arr):
    shuffle = np_arr[np.random.permutation(len(np_arr))]
    start = int(len(np_arr)*0.8)
    end = int(len(np_arr)*0.9)
    train, valid, test = shuffle[:start], shuffle[start:end], shuffle[end:]
    return train, valid, test

In [17]:
a = np.array([z for z in range(0,100)])
tr, va, te = random_split(a)

print(tr, va, te)

[50  3 23 99 ... 62 79 90 12] [31 55 53 49 38 77 47 25 32 22] [40 82 63 56 94  1 80 45 76 52]


Um nur die Aktivitäten zu benutzen vorerst data["Activities"].values benutzen.

In [18]:
def rand_split(df):
    
    train, valid, test = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])    
    print(len(train), len(valid), len(test))
    return train, valid, test

In [19]:
training, valid, test = rand_split(data)
display(training[:5])

10469 1309 1309


,Activities,Users
204885,203968 A_SUBMITTED 203969 ...,203968 112 203969 112 203970 11...
211396,245599 A_SUBMITTED 245600 A_PA...,245599 112 245600 112 245601 11...
183289,65613 A_SUBMITTED 65614 ...,65613 112 65614 112 65615 112 6...
174003,2225 A_SUBMITTED 2226 A_PARTLYSUBM...,2225 112 2226 112 2227 112 Name: org:...
178194,30471 A_SUBMITTED 30472 ...,30471 112 30472 112 30473 112 3...


In [20]:
training, valid, test = rand_split(data)
display(training[:5])

10469 1309 1309


,Activities,Users
196476,146811 A_SUBMITTED 146812 A_PART...,146811 112 146812 112 146813 11...
207080,219843 A_SUBMITTED 219844 A_PARTLY...,219843 112 219844 112 219845 112 Name...
183314,65735 A_SUBMITTED 65736 ...,65735 112 65736 112 65737 112 6...
207419,221734 A_SUBMITTED 221735 A_PART...,221734 112 221735 112 221736 11...
176266,17807 A_SUBMITTED 17808 ...,17807 112 17808 112 17809 112 1...


In [21]:
class PPDataSet():
    def __init__(self, activities, bs=64, bptt=70, shuffle=False):
        self.bs, self.bptt, self.shuffle = bs,bptt,shuffle
        total_len = sum([len(t) for t in activities])
        self.n_batch = total_len // bs
        self.a_batched = self.batchify(activities)
        #self.u_batched = self.batchify(users)

    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        a_source = self.a_batched[idx % self.bs]
        #u_source = self.u_batched[idx % self.bs]

        seq_idx = (idx // self.bs) * self.bptt
        return ((a_source[seq_idx:seq_idx+self.bptt]),
                (a_source[seq_idx+1:seq_idx+self.bptt+1]))
    
        #return ((a_source[seq_idx:seq_idx+self.bptt],u_source[seq_idx:seq_idx+self.bptt]),
         #        (a_source[seq_idx+1:seq_idx+self.bptt+1],u_source[seq_idx+1:seq_idx+self.bptt+1]))
    
    def batchify(self,d):
        texts = d
        if self.shuffle: texts = texts[torch.randperm(len(texts))]
        stream = torch.cat([torch.cat((tensor([2]), tensor(t), tensor([3]))) for t in texts])
        return stream[:self.n_batch * self.bs].view(self.bs, self.n_batch)

In [22]:
bs = 128
activities = data['Activities'].values
users = data['Users'].values
dl = DataLoader(PPDataSet(activities, shuffle=False), batch_size=bs)

ValueError: too many dimensions 'str'

In [ ]:
iter_dl = iter(dl)
xb, yb = next(iter_dl)

In [ ]:
k = sum(len(a) for a in activities)
k / len(activities)

In [ ]:
# First Batch
# xb: Input Data, yb: labels (-> Window moved by one)
# xb[0], yb[0]: Activities
# xb[1], yb[1]: Users
# xb[0][0], yb[0][0]: First Window of Activities
# xb[1][0], yb[1][0]: First Window of Users

xb[0][0], yb[0][0]

In [28]:
tokens = activity_numericalize.vocab
print(tokens, len(tokens))

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxrep', 'xxwrep', 'xxup', 'xxmaj', 'A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED', 'W_Completeren aanvraag', 'A_ACCEPTED', 'O_SELECTED', 'A_FINALIZED', 'O_CREATED', 'O_SENT', 'W_Nabellen offertes', 'O_SENT_BACK', 'W_Valideren aanvraag', 'A_REGISTERED', 'A_APPROVED', 'O_ACCEPTED', 'A_ACTIVATED', 'O_CANCELLED', 'W_Wijzigen contractgegevens', 'A_DECLINED', 'A_CANCELLED', 'W_Afhandelen leads', 'O_DECLINED', 'W_Nabellen incomplete dossiers', 'W_Beoordelen fraude'] 32


In [ ]:
text = "Jakob"

chars = tuple(set(text))

int2char = dict(enumerate(chars))

char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [ ]:
char = "a"
x = np.array([[char2int[char]]])
print(x)

### Building the model

In [23]:
def predict(model, activity, h=None, cuda=False, top_k=None):

    if cuda:
        model.cuda()
    else:
        model.cpu()

    if h is None:
        h = model.init_hidden(1)

    activity = np.array([activity])
    z = one_hot_encode(activity, len(model.vocab))
    inputs = torch.from_numpy(z)

    if cuda:
        inputs = inputs.cuda()

    h = tuple([each.data for each in h])
    out, h = model.forward(inputs, h)

    p = F.softmax(out, dim=1).data
    #print("Softmax: "+ str(p))

    if cuda:
        p = p.cpu()

    if top_k is None:
        top_activities = np.arange(len(model.vocab))
    else:
        p, top_activites = p.topk(top_k)
        top_activities = top_activities.numpy().squeeze()

    p = p.numpy().squeeze()
    #print(p)
    prediction = np.random.choice(top_activities, p=p/p.sum())

    return prediction, h

In [24]:
network.vocab

NameError: name 'network' is not defined

In [ ]:
predict(network,[0])[1][0]

In [ ]:
np.array(['A_APPROVED']).shape

In [25]:
class LSTM(nn.Module):
    
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Creating dictionaries to be able to go back and forth
        self.vocab = tokens
        self.int2word = dict(enumerate(self.vocab))
        self.word2int = {w: ii for ii, w in self.int2word.items()}
        
        # Defining the LSTM
        self.lstm = nn.LSTM(len(self.vocab), n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)
        
        # Defining a dropout layer (to reduce the chance of overfitting)
        self.dropout = nn.Dropout(drop_prob)
        
        # Defining the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.vocab))
        
        # Initializing the weights
        self.init_weights()
        
    def forward(self, x, hc):
        ''' Forward pass through the network.
            The inputs are x and the hidden/cell state "hc". '''
        
        # Get x and the new hidden state and cell memory (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        # Pass x through the dropout layer
        x = self.dropout(x)
        
        # Stack up LSTM outputs using "view" (reshaping the tensor for fc layer)
        x = x.view(x.size()[0] * x.size()[1], self.n_hidden)
        
        # Pass x through the fully connected layer
        x = self.fc(x)
        
        # Return x and the hidden state (h, c)
        return x, (h, c)
    
 
    
#    def predict(self, activity, h=None, cuda=False, top_k=None):
#        ''' Given one activity, predict the next activity.
#            Returns the predicted word and the hidden state. '''
#        
#        if cuda:
#            self.cuda()
#        else:
#            self.cpu()
#        
#        # initialize the hidden state
#        if h is None:
#            h = self.init_hidden(1)
#        
#        # creating our input by grabbing the integer of the given activity
#        # and creating a numpy array out of that before one-hot-encoding it
#        # finally convert it to a pytorch tensor
#        x = np.array([[self.word2int[activity]]])
#        x = one_hot_encode(x, len(self.vocab))
#        inputs = torch.from_numpy(x)
#        
#        if cuda:
#            inputs = inputs.cuda()
#        
#        # create the hidden state tuple
#        h = tuple([each.data for each in h])
#        out, h = self.forward(inputs, h)
#
#        # Softmax for a probability distribution
#        p = F.softmax(out, dim=1).data
#        
#        if cuda:
#            p = p.cpu()
#        
#        # Select the top_k probabilities or all if top_k = None
#        if top_k is None:
#            top_activities = np.arange(len(self.vocab))
#        else:
#            p, top_activities = p.topk(top_k)
#            top_activities = top_activities.numpy().squeeze()
#        
#        p = p.numpy().squeeze()
#        activity = np.random.choice(top_activities, p=p/p.sum())
#            
#        return self.int2word[activity], h
        
    def init_weights(self):
        ''' Initialize weights for fully connected layer with 0'''
        init_range = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
    
    def init_hidden(self, n_seqs):
        ''' Initializing hidden state '''
        
        # Create two new tensors of size n_layers * n_seqs * n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())

### Training Function

In [26]:
def train(net, train, valid, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, cuda=False, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: LSTM network
        train: data to train the network
        valid: data to validate the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of word steps per mini-batch
        lr: learning rate
        clip: gradient clipping (to prevent exploding gradient problem)
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    '''
    
    # putting the network in training mode
    net.train()
    
    # Loading Adam Optimizer and CrossEntropy as Loss-function
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    #val_idx = int(len(data)*(1-val_frac))
    #data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
        net.cuda()
    
    counter = 0
    n_words = len(net.vocab)
    
    
    train_activities = train['Activities'].values
    valid_activities = valid['Activities'].values
    #users = train['Users'].values
    
    for e in range(epochs):
        h = net.init_hidden(n_seqs)
        
        for x, y in DataLoader(PPDataSet(train_activities), batch_size=64):
            counter += 1
            
            # changing tensors to numpy so they can be one-hot-encoded
            x, y = x.numpy(), y.numpy()
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_words)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # Zero the gradients
            net.zero_grad()
            
            # Pass the input and the hidden state through forward 
            # and pass the output into the criterion function ( + reshaping the target values)
            output, h = net.forward(inputs, h)
            loss = criterion(output, targets.view(n_seqs*n_steps).type(torch.LongTensor))
            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []

                for x, y in DataLoader(PPDataSet(valid_activities), batch_size=64):
                    # changing tensors to numpy so they can be one-hot-encoded
                    x, y = x.numpy(), y.numpy()

                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_words)
                    inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output, targets.view(n_seqs*n_steps).type(torch.LongTensor))

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Training loop Safe from old Version of NB 11

In [ ]:
#def get_dls(train_ds, valid_ds, bs, **kwargs):
#    return (DataLoader(PPDataSet(train_ds, shuffle=True), batch_size=bs),
#            DataLoader(PPDataSet(valid_ds, shuffle=False), batch_size=bs))

In [ ]:
#data = DataBunch(*get_dls(train, valid, 64))

In [ ]:
#len(vocabs["concept:name"].vocab)

In [ ]:
# Initialize and print the network
vocab_size = len(vocabs["concept:name"].vocab)

network = LSTM(vocab_size, n_hidden=256, n_layers=6)
learn(network, dl, dl2, vocab_size=vocab_size, epochs=10, lr=0.001)

## learn

In [ ]:
def learn(model, train_dl, valid_dl, vocab_size, epochs=10, bs=64, lr=0.005, clip=5, cuda=False):
    ''' Training model on training data
    
        Arguments
        ---------
        
        model: Neural network
        train_dl: dataloader for training data
        valid_dl: dataloader for validation data
        vocab_size: size of the vocab of the model
        epochs: Number of epochs to train
        bs: batch size
        lr: learning rate
        clip: gradient clipping (to prevent exploding gradient problem)
        cuda: Train with CUDA on a GPU
    '''
    
    # putting the model in training mode
    print(model)
    model.train()
    
    # Loading Adam Optimizer and CrossEntropy as Loss-function
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    if cuda:
        model.cuda()

    counter = 0
    for e in range(epochs): 
        
        for x, y in iter(train_dl):
            x = x[:, 3]  # number of the desired column --> in this case the concept:name
            #y = y[:, 3]  # number of the desired column --> in this case the concept:name
            counter += 1
            
            model.zero_grad()
            inputs = one_hot_encode(x)
            targets = y[:, 3] # .type(torch.LongTensor)
            #print(inputs, inputs.size())

            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()
                
            out = model.forward(inputs)
            loss = criterion(out, targets.view(bs).type(torch.LongTensor))
            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()
            
            if counter % 2 == 0:
                val_losses = []
                
                for x, y in iter(valid_dl):
                    v_inputs, v_targets = x, y
                    
                    if cuda:
                        v_inputs, v_targets = inputs.cuda(), targets.cuda()
                    
                    v_out = model.forward(v_inputs)
                    v_loss = criterion(v_out, v_targets.view(bs).type(torch.LongTensor))
                    v_loss.backward()
                    
                    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
                    nn.utils.clip_grad_norm_(model.parameters(), clip)
                    opt.step()
                    
                    val_losses.append(v_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

#def get_dls(train_ds, valid_ds, bs, **kwargs):
#    return (DataLoader(PPDataSet(train_ds, shuffle=True), batch_size=bs),
#            DataLoader(PPDataSet(valid_ds, shuffle=False), batch_size=bs))

#data = DataBunch(*get_dls(train, valid, 64))

#len(vocabs["concept:name"].vocab)

# Initialize and print the network
vocab_size = len(vocabs["concept:name"].vocab)

network = LSTM(vocab_size, n_hidden=256, n_layers=6)
learn(network, dl, dl2, vocab_size=vocab_size, epochs=10, lr=0.001)

In [29]:
# Initialize and print the network
network = LSTM(tokens, n_hidden=256, n_layers=6)
print(network)
# set n_sequences and n_steps (n_sequences * n_steps = batch_size --> n_batches = data // batch_size)
n_seqs = 64
n_steps = 70

train(network, training, valid, epochs=1, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, print_every=10)

LSTM(
  (lstm): LSTM(32, 256, num_layers=6, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=32, bias=True)
)


ValueError: could not determine the shape of object type 'Series'

### Monitoring Validation Loss vs. Training Loss

If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:
- If your training loss is much lower than validation loss then this means the network might be overfitting. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
- If your training/validation loss are about equal then your model is underfitting. Increase the size of your model (either number of layers or the raw number of neurons per layer)


### Approximate number of parameters

The two most important parameters that control the model are n_hidden and n_layers. I would advise that you always use n_layers of either 2/3. The n_hidden can be adjusted based on how much data you have. The two important quantities to keep track of here are:

- The number of parameters in your model. This is printed when you start training.
- The size of your dataset. 1MB file is approximately 1 million characters.

These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

- I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make n_hidden larger.
- I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.


### Best models strategy

The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

### Creating Samples of Traces by using the Model

In [ ]:
def sample(net, size, prime='The', top_k=None, cuda=False):
        
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    
    h = net.init_hidden(1)
    
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ' '.join(chars)

In [ ]:
sample(network, 40, prime=["A_SUBMITTED", "A_PARTLYSUBMITTED", "A_PREACCEPTED"], cuda=True)

### Building Test Measures

In [30]:
import editdistance as ed

In [31]:
def process_data_for_suffix_prediction(test, col='Activities', startIndex=1):
    
    x, y = [], []
    for trace in test[col]:
        for i in range(startIndex, len(trace)):
            x.append(trace[:i])
            y.append(trace[i:])
            
    return x, y

In [ ]:
# x, y = process_data_for_suffix_prediction(test)
# list(zip(x, y))

In [ ]:
def process_data_for_next_step_prediction(test, col='Activities', startIndex=1):
    
    x, y = [], []
    for trace in test[col]:
        for i in range(startIndex, len(trace)):
            x.append(trace[:i])
            y.append(trace[i])
            
    return x, y

In [ ]:
x, y = process_data_for_next_step_prediction(test)
list(zip(x,y))

print(len(x) == len(y))

In [ ]:
def suffix_measure(preds, ys):
    
    summe = 0.0
    for p, y in zip(preds, ys):
        l = len(p)
        d = ed.eval(p, y)
        sim = 1 - d/l
        summe += sim
    
    return summe/len(preds)

In [ ]:
def next_step_measure(preds, ys):
    # Simple accuracy measure
    # Do I have to weight it? Check Paper!
    
    return (np.array(preds) == np.array(ys)).astype(float).mean()

In [ ]:
def predict_next_step(net, test, top_k=None, cuda=False):
    
    x, y = process_data_for_next_step_prediction(test, col='Activities', startIndex=1)

    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    h = net.init_hidden(1)
    predictions = []
    
    # First off, run through the prime characters
    for value in x:
        activities = []
        
        for i in range(len(value)):
            activity, h = net.predict([value[i]], h, cuda=cuda, top_k=top_k)
            
        activities.append(activity)
        activity, h = net.predict([activities[-1]], h, cuda=cuda, top_k=top_k)
        predictions.append(activity)
        
    return predictions

In [ ]:
preds = predict_next_step(network, test)

In [ ]:
x, y = process_data_for_next_step_prediction(test)

next_step_measure(preds, y)

In [ ]:
def predict_suffix(net, test, top_k=None, cuda=False):
    
    x, y = process_data_for_suffix_prediction(test, col='Activities', startIndex=1)
        
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    h = net.init_hidden(1)
    predictions = []
    
    # First off, run through the prime characters
    for i in range(len(x)):
        
        # Get the size of the trace to be predicted
        size = len(y[i])
        value = x[i]
        activities = []
            
        for j in range(len(value)):
            activity, h = net.predict([value[j]], h, cuda=cuda, top_k=top_k)

        activities.append(activity)

        for k in range(size-1):
            activity, h = net.predict([activities[-1]], h, cuda=cuda, top_k=top_k)
            activities.append(activity)

        print(activities)

        predictions.append(activities)
        
    return predictions

In [ ]:
%%time 

preds2 = predict_suffix(network, test)

In [ ]:
x, y = process_data_for_suffix_prediction(test, col='Activities', startIndex=1)

suffix_measure(preds2, y)

In [ ]:
def evaluation(model, data, measurement):
    predictions = []
    x, y = process_data_for_suffix_prediction(data)
    
    h = model.init_hidden(1)
    
    for i in range(0, len(x)):
        print(i)
        for j in len(0, len(x[i])):
            pred, h = model.predict(x[i], h)
            predictions.append(pred)
        
    return suffix_measure(predictions, y)

In [ ]:
x = [[8], [8, 9], [8, 9, 10]]

for i in range(0, len(x)):
    z = np.array([x[i]])
    print(z)
    
    one_hot = np.zeros((np.multiply(*z.shape), 24), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), z.flatten()] = 1.
    one_hot = one_hot.reshape((*z.shape, 24))
    print(one_hot)

In [ ]:
evaluation(network, test, measurement="suffix_measure")

In [ ]:
x, y = process_data_for_suffix_prediction(test)
model = network
preds = model.predict(x)
suffix_measure(preds, y)

In [ ]:
def next_step_measure(preds, ys):
    # Simple accuracy measure
    # Do I have to weight it? Check Paper!
    return (np.array(preds) == np.array(ys)).astype(float).mean()

In [ ]:
x, y = process_data_for_suffix_prediction(test)
model = PerfectModel(y)
preds = model.predict(x)
next_step_measure(preds,y)

### Creating a FastAi Model

In [ ]:
TextList.label_for_lm??

In [ ]:
data = (TextList.from_df(df=df, path=path, cols='concept:name')
                .split_by_idx(valid_idx=[10000,13086])
                .label_for_lm()
                .databunch())

**All das preprocessing wird durch die obige Funktion im Hintergrund durchgeführt. FastAi kodiert den Text dabei auf folgende Art und Weise:**

- xxbos: Begining of a sentence
- xxfld: Represent separate parts of a document like title, summary etc. (separate field for each one)
- xxup:  If there's something in all caps, it gets lower cased and a token called xxup will get added to it.
- xxunk: token used instead of an uncommon word.
- xxmaj: token indicates that there is capitalization of the word. “The” will be tokenized as “xxmaj the“.
- xxrep: token indicates repeated word, if you have 29 ! in a row, (i.e. xxrep 29 !)

The list of unique possible tokens is called the vocabulary. Below the first 20 unique tokens are printed in order of there frequency. The minimal frequency for a word to be added to the vocabulary is 2, to avoid getting a huge weight matrix (The default vocab size is set to 60000). Since it is easier for machines to deal with numerical values (numbers) the tokens will be replaced by their position in the vocabulary.

In [ ]:
print('The first 20 unique tokens ordered by frequency: \n\n' + str(data.vocab.itos[:20]))

In [ ]:
display(data.train_ds[0][0], data.train_ds[0][0].data[:10])

In [ ]:
data.show_batch()

In [ ]:
len(data.vocab.itos)

In [ ]:
language_model_learner??

In [ ]:
# for pretained model: pretrained_model=...
# to avoid underfitting you can specify the amount of dropout (some kind of regularization) with drop_mult=...
# arch is a Callable (?) it can be one of the following: 
# - AWD_LSTM (Merity et al.)
# - Transformer decoder (Vaswani et al.) 
# - TransformerXL (Dai et al.)

learn = language_model_learner(data=data, arch=AWD_LSTM, model_dir="/home/lahann/.fastpm/dl_models")

FastAi provides a function called **lr_find()** to search through a range of learning rates to find the optimum one for the underlying dataset. Learning rate finder will increase the learning rate after each mini-batch. Eventually, the learning rate is too high that loss will get worse. Now look at the plot (**recorder.plot()**) of learning rate against loss and determine the lowest point (around 1e+00 for the plot below) and go back by one magnitude and choose that as a learning rate (something around 1e-01).

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

**Fit one cycle:**

What is fit_one_cycle()? It is one cycle of learning rate, start low, go up, and then go down again.
When we call fit_one_cycle, we are actually passing in a maximum learning rate. The learning starts slow and it increases about half the time and then it decreases about half the time. As you get close to the final answer you need to anneal your learning rate to hone in on it. The motivation behind this is that during the middle of learning when learning rate is higher, the learning rate works as regularization method and keeps the network from over-fitting. This helps the network to avoid steep areas of loss and land better flatter minima. Please refer to this paper (https://arxiv.org/pdf/1803.09820.pdf) by Leslie smith which talks in great detail about neural network hyper-parameter tuning and you can find most of these ideas implemented in fastai.

In [ ]:
#learn.fit_one_cycle(10, 1e+00)

In [ ]:
learn.fit_one_cycle(10, 1e-02)

In [ ]:
#learn.fit_one_cycle(10, 1e-02)

In [ ]:
learn.model

In [ ]:
# list(learn.model.parameters())  # for getting the weights of the trained model

In [ ]:
learn.get_preds??

In [ ]:
preds, y, losses= learn.get_preds(with_loss=True)

In [ ]:
preds.shape,y.shape

In [ ]:
interpret = ClassificationInterpretation(learn, preds, y, losses)

In [ ]:
validation = interpret.y_true

In [ ]:
interpret.pred_class

In [ ]:
validationset = interpret.data.valid_ds
validationset[0]

In [ ]:
def random_split(np_arr):
    shuffle = np_arr[np.random.permutation(len(np_arr))]
    start = int(len(np_arr)*0.8)
    end = int(len(np_arr)*0.9)
    train, test, valid = shuffle[:start], shuffle[start:end], shuffle[end:]
    return train, test, valid

In [ ]:
train, test, valid = random_split(df.values)
display(valid[:,2])

In [ ]:
def eval_next_step_prediction(test,model,start_sequenze=1):
    sum=0
    acc=0
    for t in test:
        for a in range(start_sequenze,len(t)-1):
            pred=model.predict(t[:a])
            print(pred)
            if pred==t[a]: acc+=1
            sum+=1
    return acc/sum

eval_next_step_prediction(validation, learn)

In [ ]:
# dep_var = 'salary'
# categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
# continuous = []
# preprocesses = [FillMissing, Categorify, Normalize]

In [ ]:
# test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=categorical, cont_names=continuous)

In [ ]:
# data = (TabularList.from_df(df, path=path, cat_names=categorical, cont_names=continuous, procs=preprocesses)
#        .split_by_idx(list(range(800, 1000)))
#        .label_from_df(cols=dep_var)
#        .add_test(test, label=0)
#        .databunch())

In [ ]:
# data.show_batch(rows=10)

In [ ]:
# tabular_learner??

In [ ]:
# learn = tabular_learner(data, layers=[20, 10], metrics=accuracy)
# learn.fit_one_cycle(3, 1e-1)

In [ ]:
# learn.model

In [ ]:
# path2 = untar_data(URLs.BPIC_2017)

In [ ]:
# next step prediction

### Playing around

In [149]:
len(tp.vocabs["concept:name"].vocab)

32

In [150]:
padding_index = 1
vocab_size = len(tp.vocabs["concept:name"].vocab)

In [151]:
tst_model = get_language_model(vocab_size, 300, 300, 2, padding_index)
tst_model = tst_model

In [152]:
tst_model

SequentialRNN(
  (0): AWD_LSTM(
    (emb): Embedding(32, 300, padding_idx=1)
    (emb_dp): EmbeddingDropout(
      (emb): Embedding(32, 300, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(300, 300, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (output_dp): RNNDropout()
    (decoder): Linear(in_features=300, out_features=32, bias=True)
  )
)

In [155]:
z = tst_model(xb[:, 3].long())
z

(tensor([[ 0.0004, -0.0122, -0.0210,  ...,  0.0006,  0.0540,  0.0224],
         [ 0.0022, -0.0157, -0.0268,  ..., -0.0063,  0.0502,  0.0171],
         [ 0.0013, -0.0147, -0.0332,  ..., -0.0107,  0.0465,  0.0149],
         ...,
         [ 0.0097, -0.0136, -0.0041,  ...,  0.0548,  0.0716,  0.0674],
         [ 0.0084, -0.0158, -0.0058,  ...,  0.0622,  0.0696,  0.0680],
         [ 0.0048, -0.0181, -0.0062,  ...,  0.0636,  0.0680,  0.0653]],
        grad_fn=<AddmmBackward>),
 [tensor([[[ 1.0967e-02,  2.2706e-02,  1.3375e-02,  ..., -1.2531e-02,
             1.7527e-02, -1.9572e-02],
           [ 1.1744e-02,  1.9288e-02,  5.2221e-03,  ..., -2.7696e-02,
             2.3785e-02, -4.0682e-02],
           [ 1.6345e-02,  2.6600e-02,  1.1789e-02,  ..., -2.0935e-02,
             2.4581e-02, -3.1756e-02],
           ...,
           [ 3.0383e-03,  4.4811e-02,  2.6414e-02,  ..., -3.7662e-02,
             2.0990e-02, -9.7780e-03],
           [-3.5917e-03,  6.2961e-02,  9.6876e-03,  ..., -2.5391e-02,
   

In [169]:
z[0].shape, 128*70

(torch.Size([8960, 32]), 8960)

In [170]:
def cross_entropy_flat(input, target):
    bs, sl = target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.view(bs * sl))

In [199]:
def accuracy_flat(input, target):
    bs, sl = target.size()
    return accuracy(input.view(bs * sl, -1), target.contiguous().view(bs * sl))

In [193]:
decoded, raw_outputs, outputs = z

In [194]:
xb[:, 3].size(), decoded.size()

(torch.Size([128, 70]), torch.Size([8960, 32]))

In [195]:
yb[:, 3].size()

torch.Size([128, 70])

In [201]:
accuracy_flat(decoded, yb[:, 3].long()) # funktioniert nur mit contiguous in accuracy und .long in der eingabe

128 70


tensor(0.0190)

In [202]:
cross_entropy_flat(decoded,yb[:, 3].long())

tensor(3.4703, grad_fn=<NllLossBackward>)

In [203]:
bs,sl = yb[:, 3].size()
bs,sl

(128, 70)

In [205]:
len(yb[:, 3].contiguous().view(bs * sl))

8960

In [206]:
decoded

tensor([[ 0.0004, -0.0122, -0.0210,  ...,  0.0006,  0.0540,  0.0224],
        [ 0.0022, -0.0157, -0.0268,  ..., -0.0063,  0.0502,  0.0171],
        [ 0.0013, -0.0147, -0.0332,  ..., -0.0107,  0.0465,  0.0149],
        ...,
        [ 0.0097, -0.0136, -0.0041,  ...,  0.0548,  0.0716,  0.0674],
        [ 0.0084, -0.0158, -0.0058,  ...,  0.0622,  0.0696,  0.0680],
        [ 0.0048, -0.0181, -0.0062,  ...,  0.0636,  0.0680,  0.0653]],
       grad_fn=<AddmmBackward>)

In [281]:
#(torch.argmax(decoded, dim=1) == yb[:, 3].contiguous().view(bs * sl).cuda()).float().mean()

In [210]:
decoded.shape

torch.Size([8960, 32])

In [211]:
len(raw_outputs), len(outputs)

(2, 2)

In [212]:
[o.size() for o in raw_outputs], [o.size() for o in outputs]

([torch.Size([128, 70, 300]), torch.Size([128, 70, 300])],
 [torch.Size([128, 70, 300]), torch.Size([128, 70, 300])])